In [1]:
import numpy as np
import logging
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import heart_disease.plotting as hdp
import heart_disease.preprocessing as hdpp
import heart_disease.parameter_optimization as hdpo
from heart_disease.globals import data_path, output_path

import seaborn as sns
import pandas as pd
from copy import deepcopy as cp
%load_ext autoreload

from heart_disease.globals import code_path, raw_data_path, data_path, output_path
from heart_disease.

%autoreload 2

In [2]:
LOG_FMT = '%(asctime)s|%(name)s|%(levelname)s|%(message)s'
logging.basicConfig(level=logging.INFO, format=LOG_FMT)
logger = logging.getLogger(__name__)

In [3]:
# Get Metadata
meta_data_df = pd.read_csv(data_path+'meta_data.csv')
meta_data_df.head()

,name,datatype,description,id
0,age,numeric,age in years,3
1,sex,bool,sex (1 = male; 0 = female),4
2,cp,categorical,chest pain type\r -- Value 1: typical angina...,9
3,trestbps,numeric,resting blood pressure (in mm Hg on admission ...,10
4,chol,numeric,serum cholestoral,12


In [4]:
# Import Raw Data
data_file_name_str = 'train_validation.cleveland.csv'
data_df = hdpp.load_data(data_path+data_file_name_str)
data_df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,ispos_truth
pat_id,,,,,,,,,,,,,,
162,54.0,0.0,3.0,110.0,214.0,0.0,0.0,158.0,0.0,1.6,2.0,0.0,3.0,False
18,48.0,0.0,3.0,130.0,275.0,0.0,0.0,139.0,0.0,0.2,1.0,0.0,3.0,False
100,45.0,1.0,4.0,115.0,260.0,0.0,2.0,185.0,0.0,0.0,1.0,0.0,3.0,False
78,48.0,1.0,2.0,130.0,245.0,0.0,2.0,180.0,0.0,0.2,2.0,0.0,3.0,False
262,60.0,0.0,1.0,150.0,240.0,0.0,0.0,171.0,0.0,0.9,1.0,0.0,3.0,False


In [30]:
clean_df = hdpp.DataCleaner(data_path+'meta_data.csv').get_clean_dataframe(data_df)

In [31]:
clean_df.head()

,age,trestbps,chol,thalach,oldpeak,sex,fbs,exang,cp_is1,cp_is2,...,slope_is1,slope_is2,slope_is3,ca_is0,ca_is1,ca_is2,ca_is3,thal_is3,thal_is6,thal_is7
0,-0.056774,-1.287754,-0.616740,0.327936,0.449032,-1.467235,-0.426956,-0.667727,-0.293470,-0.412436,...,-0.944281,1.107019,-0.29347,0.856052,-0.531494,-0.405096,-0.256374,0.919526,-0.225668,-0.833085
1,-0.717684,-0.115660,0.517089,-0.503080,-0.715885,-1.467235,-0.426956,-0.667727,-0.293470,-0.412436,...,1.059007,-0.903327,-0.29347,0.856052,-0.531494,-0.405096,-0.256374,0.919526,-0.225668,-0.833085
2,-1.048139,-0.994731,0.238279,1.508854,-0.882301,0.681554,-0.426956,-0.667727,-0.293470,-0.412436,...,1.059007,-0.903327,-0.29347,0.856052,-0.531494,-0.405096,-0.256374,0.919526,-0.225668,-0.833085
3,-0.717684,-0.115660,-0.040532,1.290165,-0.715885,0.681554,-0.426956,-0.667727,-0.293470,2.424621,...,-0.944281,1.107019,-0.29347,0.856052,-0.531494,-0.405096,-0.256374,0.919526,-0.225668,-0.833085
4,0.604136,1.056434,-0.133469,0.896526,-0.133427,-1.467235,-0.426956,-0.667727,3.407508,-0.412436,...,1.059007,-0.903327,-0.29347,0.856052,-0.531494,-0.405096,-0.256374,0.919526,-0.225668,-0.833085


In [32]:
clean_df.columns

Index(['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'sex', 'fbs', 'exang',
       'cp_is1', 'cp_is2', 'cp_is3', 'cp_is4', 'restecg_is0', 'restecg_is1',
       'restecg_is2', 'slope_is1', 'slope_is2', 'slope_is3', 'ca_is0',
       'ca_is1', 'ca_is2', 'ca_is3', 'thal_is3', 'thal_is6', 'thal_is7'],
      dtype='object')